## HTML post-process

`Typora` generates an HTML file with inline CSS styling, with the selected theme at the moment of the export.
The cascade is as follows:

1. Base CSS from Typora, extracted to its own CSS file: `typora.base.css`
1. Selected base theme, in our case the files `monospace-dark.css` or `monospace.css`
1. Custom common CSS for Typora, called `base.user.css` in the Typora themes folder.
1. Custom CSS for each theme, using files with ".user" postfix: `monospace-dark.user.css` or `monospace.user.css`

By using media queries we can implement both themes to be in sync with the viewer local config,
and force any of the 2 themes for `@media print`.

The expected html in `<head>` should be something like this:

```html
<link href='typora.base.css' rel='stylesheet' type='text/css' />
<link href='monospace-dark.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference), print"/>
<link href='monospace.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: light)"/>
<link href='base.user.css' rel='stylesheet' type='text/css' />
<link href='monospace-dark.user.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference), print"/>
<link href='monospace.user.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: light)" />
```

A manual switch (hidden for print) is also added to toggle between themes, by just inserting 2 lines:

```html
<!-- on head -->
<script type="module" src="https://googlechromelabs.github.io/dark-mode-toggle/src/dark-mode-toggle.mjs"></script>
```

```html
<!-- at the end of body -->
<aside>
    <dark-mode-toggle id="dark-mode-toggle-1" appearance="toggle" legend="dark mode" dark="on" light="off" remember="remember"></dark-mode-toggle>
</aside>
```


In [2]:
from pathlib import Path
import re

p_base = Path("../resumes")

# input (generated on Typora > export to HTML)
p_html_typora = p_base / "resume-eugenio-panadero.html"
base_name = p_html_typora.name.split('.')[0]

# output
rel_path_css = "resources/"
#p_dark = p_html_typora.parent / f"{base_name}-dark-theme.html"
#p_light = p_html_typora.parent / f"{base_name}-light-theme.html"
p_index = p_base.parent / f"index.html"

# Dark mode switch (CSS to position the `aside` custom tag is included in base.user.css)
line_import_dark_mode_toggle = '<script type="module" src="https://googlechromelabs.github.io/dark-mode-toggle/src/dark-mode-toggle.mjs"></script>'
line_component_aside = '<aside><dark-mode-toggle id="dmtoggle-1" appearance="toggle" legend="dark mode" dark="on" light="off" remember="remember"></dark-mode-toggle></aside>'

# head metadata
meta_desc = '<meta name="description" content="Python Senior Developer (12+ years of experience) interested in web apps, data analysis & ML, IoT, home automation, and the energy & electrical industrial sectors" >'
meta_language = '<meta http-equiv="Content-Language" content="en" >'
meta_distr = '<meta name="distribution" content="global" />'
meta_no_translate = '<meta name="google" content="notranslate" />'


def _css_link(rel_path, name, media_query=None):
    link_line = f"<link href='{rel_path}{name}' rel='stylesheet' type='text/css' "
    if media_query is not None:
        link_line += f"media='{media_query}'"
    return link_line + " />"


def _make_new_header(rel_path, print_dark=True, add_toggle=True, add_desc=True):
    if print_dark:
        q_dark = "screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference), print"
        q_light = "screen and (prefers-color-scheme: light)"
    else:
        q_dark = "screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference)"
        q_light = "screen and (prefers-color-scheme: light), print"
    
    css_files_media = {
        "typora.base.css": None,
        "monospace-dark.css": q_dark,
        "monospace.css": q_light,
        "base.user.css": None,
        "monospace-dark.user.css": q_dark,
        "monospace.user.css": q_light,        
    }
    lines = [""]
    if add_desc:
        lines += [meta_desc, meta_language, meta_distr, meta_no_translate]        
    
    lines += [_css_link(rel_path, name, media_query) for name, media_query in css_files_media.items()]
    if add_toggle:
        lines.append(line_import_dark_mode_toggle)
    return "\n".join(lines)
    

def extract_css_to_links(html_text, rel_path="../resources/", print_dark=True, add_toggle=True):
    """
    Remove inline CSS from header and link to CSS files instead.
    
    * Enable light & dark modes depending on client
    * Set dark/light mode on media print
    * Add a web component to toggle manually (makes media print set to light, where toggle is hidden)
    """
    html_imports = re.sub(
        r"(<style .*</style>)",
        _make_new_header(rel_path, print_dark, add_toggle, add_desc=True),
        html_text,
        flags=re.DOTALL,
    )
    if add_toggle:
        return re.sub(r"(</body>)", f"{line_component_aside}\n" + r"\1", html_imports)
        
    return html_imports



# Load original & produce html files with selectable theme & +
html_orig = p_html_typora.read_text()

#p_dark.write_text(extract_css_to_links(html_orig, rel_path_css, print_dark=True, add_toggle=False))
#p_light.write_text(extract_css_to_links(html_orig, rel_path_css, print_dark=False, add_toggle=True))

# index.html generation
p_index.write_text(
    extract_css_to_links(html_orig, rel_path_css, print_dark=False, add_toggle=True)
)


17315